In [ ]:
from utils import *
import warnings
warnings.filterwarnings('ignore')

mm = .1/2.54

In [ ]:
def get_ar_circle(radius):
    radius_clip = {3: 2.54, 4:3.53, 5: 4.49, 6: 5.52, 7: 6.52, 8: 7.49, 9: 8.52, 10: 9.49, 11: 10.52, 12: 11.49, 13: 12.49, 14: 13.51, 15: 14.49, 16: 15.52, 17: 16.46, 18: 17.49, 19: 18.49, 20: 19.47, 21: 20.49, 22: 21.49, 23: 22.49, 24: 23.49, 25: 24.43}[radius]
    gdf_circle = gpd.GeoDataFrame({}, geometry=gpd.GeoDataFrame({}, geometry=gpd.points_from_xy([0], [0])).buffer(radius_clip), crs="epsg:4326")
    da_circle = xr.DataArray(np.ones((radius * 2 + 1, radius * 2 + 1)), coords={"y": np.arange(-radius, radius + 1), "x": np.arange(-radius, radius + 1)})\
        .rio.write_crs("epsg:4326")\
        .rio.clip(gdf_circle.geometry, all_touched=True, drop=False)\
        .fillna(0).astype(np.uint8)
    return da_circle, da_circle.values

In [ ]:
radius = 10
da_circle, ar_circle = get_ar_circle(radius)
pixel_num = ar_circle.sum() 
pixel_area = pixel_num * 1e6

# identify loss type

In [ ]:
years = np.arange(2002, 2023)

In [ ]:
dic_region = {"Northern Africa and Western Asia": "N. Africa W. Asia", "Sub-Saharan Africa": "Sub-Saharan Africa", "Central Asia and Russian Federation": "C. Asia", "Eastern Asia": "E. Asia", "Southern Asia": "S. Asia", "Southeastern Asia": "S. Asia", "Northern America": "N. America", "Latin America and the Caribbean": "Latin America", "Western Europe": "W. Europe", "Eastern and South-Eastern Europe": "E. Europe", "Oceania and Australia": "Oceania",}
gdf_countries = gdf_world.reset_index(drop=True).reset_index().rename(columns={"index": "idx"})
gdf_countries["regi_short"] = gdf_countries["regi_pnas"].map(dic_region)
country_to_region = gdf_countries.set_index("name_long")["regi_short"].to_dict()

In [ ]:
regions = ['C. Asia', 'E. Asia', 'S. Asia', 'N. Africa W. Asia', 'Sub-Saharan Africa', 'N. America', 'Latin America', 'Oceania', 'E. Europe', 'W. Europe']
regions_to_color = dict(zip(regions, plt.get_cmap("tab20")(np.linspace(0, 1, len(regions)))))

In [ ]:
def ttest_2_ttest_class(df_ttest_merge):
    df_ttest_class_result = df_ttest_merge.map(lambda x: x<0)
    ttest_cols = ["CDS", "CPD", "FS", "FD", "GS", "GD"]

    df_ttest_class_result.columns = ttest_cols
    def extract_type(type_lst):
        len_ = len(type_lst)
        match len_:
            case 0:
                return "NA"
            case 1:
                return type_lst[0]
            case _:
                return "MA"

    df_ttest_class_result.loc[:, "class_"] = df_ttest_class_result.apply(lambda x: extract_type(np.array(ttest_cols)[x]), axis=1)
    df_ttest_class_result = df_ttest_class_result.reset_index()
    
    df_ttest_class_result = df_ttest_class_result.query("country != 'China'")
    return df_ttest_class_result

In [ ]:
df_ttest_merge = pd.read_csv(path_data / f"PSM_results/ttest/{radius}km.csv", index_col=[0, 1])
df_ttest_class_result = ttest_2_ttest_class(df_ttest_merge)
df_ttest_class_result.set_index(["year", "country"])[["class_"]].to_csv(path_data / f"PSM_results/ttest_class/{radius}.csv")

# NPP loss

In [ ]:
def get_df_dif_matched_all(radius):
    df_matched_m_lst = []
    df_diff_m_lst = []
    for year in range(2002, 2023):
        df_matched_m_lst.append(pd.read_csv(path_data / f"PSM_results/matched/{radius}km_{year}.csv", index_col=0))
        df_diff_m_lst.append(pd.read_csv(path_data / f"PSM_results/diff/{radius}km_{year}.csv", index_col=0))
    df_diff_m_all = pd.concat(df_diff_m_lst)
    df_matched_m_all = pd.concat(df_matched_m_lst)
    
    for luc_ in ["crop", "forest", "grass"]:
        df_crop_npp_diff_country = pd.read_csv(path_data / f"npp/{luc_}_npp_diff.csv").rename(columns={"name_long": "country"})
        df_crop_npp_diff_country = df_crop_npp_diff_country.melt(id_vars="country", var_name="year", value_name=f"{luc_}_npp_diff_country")\
            .assign(year=lambda _df: _df["year"].astype(int))

        df_matched_m_all = df_matched_m_all.merge(df_crop_npp_diff_country, on=["country", "year"], how="left")
        df_matched_m_all.loc[:, f"{luc_}_npp_change_n"] = df_matched_m_all[[f"{luc_}_npp_change_n", f"{luc_}_npp_diff_country"]].bfill(axis=1).iloc[:, 0].values
        df_matched_m_all = df_matched_m_all.drop(columns=[f"{luc_}_npp_diff_country"])
        
    df_diff_m_all = df_diff_m_all.query("country != 'China'")
    df_matched_m_all = df_matched_m_all.query("country != 'China'")
    return df_diff_m_all, df_matched_m_all

In [ ]:
def cal_npp_loss(df_matched_m_all, radius):
    df_ttest_merge = pd.read_csv(path_data / f"PSM_results/ttest/{radius}km.csv", index_col=[0, 1])
    df_ttest_class_result = ttest_2_ttest_class(df_ttest_merge)

    _, ar_circle = get_ar_circle(radius)
    pixel_num = ar_circle.sum() 
    pixel_area = pixel_num * 1e6
    
    df_matched_m_all1 = df_matched_m_all[[
        "crop_cy", "crop_ly", "grass_ly", "grass_cy", "forest_ly", "forest_cy", 
        "crop_npp_cy", "crop_npp_ly", "grass_npp_ly", "grass_npp_cy", "forest_npp_ly", "forest_npp_cy", 
        "country", "year", "ratio"
    ]].copy()\
        .assign(crop_change=df_matched_m_all["crop_change"] - df_matched_m_all["crop_change_n"])\
        .assign(crop_npp_change=df_matched_m_all["crop_npp_change"] - df_matched_m_all["crop_npp_change_n"])\
        .assign(forest_change=df_matched_m_all["forest_change"] - df_matched_m_all["forest_change_n"])\
        .assign(forest_npp_change=df_matched_m_all["forest_npp_change"] - df_matched_m_all["forest_npp_change_n"])\
        .assign(grass_change=df_matched_m_all["grass_change"] - df_matched_m_all["grass_change_n"])\
        .assign(grass_npp_change=df_matched_m_all["grass_npp_change"] - df_matched_m_all["grass_npp_change_n"])\
        .reset_index()\
        .merge(df_ttest_class_result, on=["year", "country"])\
        .set_index("index")
        
    df_matched_m_all1_crop_d = df_matched_m_all1.query("CPD == True")
    df_matched_m_all1_crop_s = df_matched_m_all1.query("CDS == True")
    df_matched_m_all1_forest_d = df_matched_m_all1.query("FD == True")
    df_matched_m_all1_forest_s = df_matched_m_all1.query("FS == True")
    df_matched_m_all1_grass_d = df_matched_m_all1.query("GD == True")
    df_matched_m_all1_grass_s = df_matched_m_all1.query("GS == True")
    
    degradation_forest_df = df_matched_m_all1_forest_d.query("forest_npp_change<0")\
        .assign(degradation_forest_npp=lambda _df: _df["forest_cy"] * _df["forest_npp_change"] * pixel_area * _df["ratio"])\
        .groupby(["year", "country"], as_index=False)["degradation_forest_npp"].sum()\
        .set_index(["year", "country"])
    degradation_grass_df = df_matched_m_all1_grass_d.query("grass_npp_change<0")\
        .assign(degradation_grass_npp=lambda _df: _df["grass_cy"] * _df["grass_npp_change"] * pixel_area * _df["ratio"])\
        .groupby(["year", "country"], as_index=False)["degradation_grass_npp"].sum()\
        .set_index(["year", "country"])

    shrink_forest_df = df_matched_m_all1_forest_s.query("forest_change<0")\
        .assign(shrink_forest_npp=lambda _df: _df["forest_change"] * _df["forest_npp_cy"] * pixel_area * _df["ratio"])\
        .groupby(["year", "country"], as_index=False)["shrink_forest_npp"].sum()\
        .set_index(["year", "country"])
    shrink_grass_df = df_matched_m_all1_grass_s.query("grass_change<0")\
        .assign(shrink_grass_npp=lambda _df: _df["grass_change"] * _df["grass_npp_cy"] * pixel_area * _df["ratio"])\
        .groupby(["year", "country"], as_index=False)["shrink_grass_npp"].sum()\
        .set_index(["year", "country"])
        
    degradation_crop_df = df_matched_m_all1_crop_d.query("crop_npp_change<0")\
        .assign(degradation_crop_npp=lambda _df: _df["crop_cy"] * _df["crop_npp_change"] * pixel_area * _df["ratio"])\
        .groupby(["year", "country"], as_index=False)["degradation_crop_npp"].sum()\
        .set_index(["year", "country"])

    abandon_crop_df = df_matched_m_all1_crop_s.query("crop_change<0")\
        .assign(abandon_crop_npp=lambda _df: _df["crop_change"] * _df["crop_npp_cy"] * pixel_area * _df["ratio"])\
        .groupby(["year", "country"], as_index=False)["abandon_crop_npp"].sum()\
        .set_index(["year", "country"])
        
    degradation_vegetation = degradation_forest_df.join(degradation_grass_df, how="outer")
    shrink_vegetation = shrink_forest_df.join(shrink_grass_df, how="outer")
    loss_vegetation = degradation_vegetation.join(shrink_vegetation, how="outer").fillna(0)
    loss_crop_df = degradation_crop_df.join(abandon_crop_df, how="outer")
    loss_df = loss_vegetation.join(loss_crop_df, how="outer").fillna(0) * -1
    loss_df = loss_df.reset_index().assign(region=lambda _df: _df["country"].map(country_to_region)).set_index(["year", "region", "country"])
    return loss_df

In [ ]:
df_diff_m_all, df_matched_m_all = get_df_dif_matched_all(radius)

df_voronoi_ratio = pd.read_csv(path_data / f"PSM/voronoi/{radius}km.csv", index_col=0)
df_matched_m_all = df_matched_m_all.merge(df_voronoi_ratio, on=["year", "idx"])

loss_df = cal_npp_loss(df_matched_m_all, radius)
(path_data / "PSM_results/loss").mkdir(exist_ok=True)
out_file = path_data / "PSM_results/loss/NPPw.csv"
loss_df.to_csv(out_file)